In [ ]:
import pandas as pd
import numpy as np

In [ ]:
DATA_FOLDER = "./cedas2025_material/data/"

In [ ]:
df = pd.read_parquet(DATA_FOLDER + "chargecurves_train.parquet")

In [ ]:
def prep_data(df: pd.DataFrame) -> None:
    # TODO: figure out what to do with soc in prediction
    """
    train: soc, nominal_power, location_id, sub_id
    label: power
    """
    df = df.groupby("id").agg(list)
    sessions = []
    labels = []
    for i, row in df.drop("timestamp", axis=1).iterrows():
        soc, power, nominal_power, location_id, sub_id = row
        session = np.empty((40,4))
        label = np.empty((40))
        session_length = len(soc)
        for i in range(session_length):
            session[i] = np.array([soc[i], nominal_power[i], location_id[i], sub_id[i]])
            label[i] = power[i]
    
        sessions.append(session)
        labels.append(label)
        
    
    sessions = np.array(sessions)
    labels = np.array(labels)

    
    # np.save("train_data.npy", sessions)
    return sessions, labels

In [ ]:
sess, lab = prep_data(df)

In [ ]:
sess.shape

In [ ]:
lab.shape